In [2]:
import os
file_path =  "../box2d/ets/TimeOfImpact.ets"
# , project_name: box2d
project_name = os.path.basename(os.path.dirname(file_path))
print(project_name)

ets


## Model 

In [4]:
import logging
import requests
from openai import OpenAI

def generate(query, system_prompt='vvYou are a helpful AI assistant', base_url='https://xiaoai.plus/v1', model='gpt-4o-mini', retries=3):

    if model == 'arktsLLM':
        for attempt in range(retries):
            try:
                data = {
                    "model": model,
                    "prompt": query,
                    "stream": False
                }
                response = requests.post('http://localhost:11434/api/generate', json=data)
                return response.json()['response']

            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed with error: {e}")
                if attempt + 1 == retries:
                    raise

    else: 
        if base_url == 'https://xiaoai.plus/v1':
            api_key="sk-Y7BDSlT4EKjhVfPpA603Bb0cC549424b9d1734262f6fE6C0"
        else:
            api_key="sk-bee3f43476e04c0ba5a0eee09961f325"
            
        client = OpenAI(
            base_url=base_url,
            api_key=api_key
        )

        for attempt in range(retries):
            try:
                if 'o1-mini' in model:
                    # 如果query是list，则认为是conversation
                    if isinstance(query, list):
                        chat_completion = client.chat.completions.create(
                            model=model,
                            messages=query
                        )
                    else:
                        chat_completion = client.chat.completions.create(
                            model=model,
                            messages=[
                                {"role": "user", "content": query}
                            ],
                    )
                else:
                    # 如果query是list，则认为是conversation
                    if isinstance(query, list):
                        chat_completion = client.chat.completions.create(
                            model=model,
                            messages=query
                        )
                    else:
                        chat_completion = client.chat.completions.create(
                        model=model,
                        messages=[
                            {"role": "system", "content": system_prompt},
                            {"role": "user", "content": query}
                        ],
                    )
                return chat_completion.choices[0].message.content
            except Exception as e:
                logging.error(f"Attempt {attempt + 1} failed with error: {e}")
                if attempt + 1 == retries:
                    raise

## RQ1 Defect Recognition

Here, we use few-shot learning to recognize defects in the steel surface. We give the rules dictionary to the model and let it learn the rules. 

The key of the rules dictionary is the defect type, and the value include the defect description and an example code. The model will learn the rules from the example code and use the rules to recognize the defects.

In [5]:
import os

def parse_ets_file(file_path):
    description = []
    code_example = ""
    in_comment = True

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            stripped_line = line.strip()
            if in_comment and stripped_line.startswith('//'):
                description.append(stripped_line[2:].strip())
            else:
                in_comment = False
                code_example += stripped_line + " "

    return {
        'description': ' '.join(description),
        'defect_code_example': code_example.strip()
    }

# 新增函数用于获取正例数据
def get_positive_example(file_path):
    code_example = ""
    in_comment = False  # 正例不需要提取注释部分
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            stripped_line = line.strip()
            code_example += stripped_line + " "
    return code_example.strip()

# 修改后的规则构建函数，包含正例数据
def build_rules_dict(negative_dir, positive_dir):
    rules = {}
    for filename in os.listdir(negative_dir):
        if filename.endswith('.ets'):
            rule_name = os.path.splitext(filename)[0]
            rule_key = f"@performance/{rule_name}"
            negative_file_path = os.path.join(negative_dir, filename)
            rule_data = parse_ets_file(negative_file_path)

            # 查找对应的正例文件
            positive_file_path = os.path.join(positive_dir, filename)
            if os.path.exists(positive_file_path):
                positive_example = get_positive_example(positive_file_path)
                rule_data['positive_code_example'] = positive_example
            else:
                rule_data['positive_code_example'] = None  # 没有正例时返回 None

            rules[rule_key] = rule_data
    return rules

negative_directory = '../pages/negative'
positive_directory = '../pages/positive'
rules = build_rules_dict(negative_directory, positive_directory)

print(rules)


{'@performance/sparse-array-check': {'description': '建议避免使用稀疏数组。  根据ArkTS高性能编程实践，建议修改。', 'defect_code_example': 'let count = 100000; let result: number[] = new Array(count); result = new Array(); result[9999] = 0;', 'positive_code_example': 'let index = 3; let result: number[] = []; result[index] = 0;'}, '@performance/hp-arkui-use-row-column-to-replace-flex': {'description': '建议使用Column/Row替代Flex。  通用丢帧场景下，建议优先修改。', 'defect_code_example': "@Entry @Component struct MyComponent { build() { // Flex Nesting Flex({ direction: FlexDirection.Column }) { Text('Replace Flex with Column/Row') .fontSize(12) .height('16') .margin({ top: 5, bottom: 10 }) Flex().width(300).height(200).backgroundColor(Color.Pink) Flex().width(300).height(200).backgroundColor(Color.Yellow) Flex().width(300).height(200).backgroundColor(Color.Grey) Flex().width(300).height(200).backgroundColor(Color.Pink) Flex().width(300).height(200).backgroundColor(Color.Yellow) Flex().width(300).height(200).backgroundColor(Color.Grey

After getting the rules, we construct a few-shot learning prompt for the model. The prompt includes the rules and the examples of the defects. The model will learn the rules from the prompt and recognize the defects.

In [6]:
print(rules.keys())

dict_keys(['@performance/sparse-array-check', '@performance/hp-arkui-use-row-column-to-replace-flex', '@performance/hp-arkui-no-state-var-access-in-loop', '@performance/hp-arkui-use-reusable-component', '@performance/hp-arkui-avoid-empty-callback', '@performance/hp-arkui-no-stringify-in-lazyforeach-key-generator', '@performance/hp-arkui-load-on-demand', '@performance/hp-arkui-use-id-in-get-resource-sync-api', '@performance/hp-arkui-replace-nested-reusable-component-by-builder', '@performance/hp-arkui-use-onAnimationStart-for-swiper-preload', '@performance/hp-arkui-suggest-use-effectkit-blur', '@performance/hp-arkui-image-async-load', '@performance/number-init-check', '@performance/lottie-animation-destroy-check', '@performance/hp-arkui-no-func-as-arg-for-reusable-component', '@performance/multiple-associations-state-var-check', '@performance/hp-arkui-use-transition-to-replace-animateto', '@performance/typed-array-check', '@performance/hp-arkui-reduce-pan-gesture-distance', '@performanc

In [7]:
print(len(rules))

46


### 识别该文件的所有缺陷

In [30]:
# 合并rules_prompt和sys_prompt中的规则说明部分
system_prompt = f"""
I am a code analyzer specialized in detecting performance defects in ArkTS code. I will carefully analyze the code line by line from top to bottom to identify any performance issues based on the rules you provided.

When you give me an ArkTS file, I will:
1. Read through the code sequentially from the first line to the last line
2. For each line, check if it violates any of the performance rules
3. If a defect is found, I will record:
   - The specific rule that was violated
   - The rule's description
   - The line number where the defect occurs
   - The problematic code snippet
   - Why this code violates the rule based on the rule's description

Please provide the code you want me to analyze. I will return the results in the following JSON format:

[
  {{
    "rule": "@performance/hp-arkui-use-reusable-component",
    "description": "避免在for、while等循环逻辑中频繁读取状态变量。通用丢帧场景下，建议优先修改。", 
    "line": 5,
    "defect_snippet": "<problematic_code>"
  }}
]

Notes:
- Results will be ordered by line number (ascending)!!!!
- Code snippets will use \\n for newlines
- I will detect ALL defects in the code, not just the first one found
- If no defects are found, I will return an empty array []
- I will only output valid JSON without any additional text

Here are the rules and their descriptions:
"""

for rule, details in rules.items():
    system_prompt += f"Rule: {rule}\nDescription: {details['description']}\nDefect Code Example: \n{details['defect_code_example']}\n Positive Code Example without this defect: \n{details['positive_code_example']}\n\n"



In [31]:
import json
import re

def handle_vul_type_res(res):
    cleaned_res = re.sub(r'^```json\s*', '', res, flags=re.MULTILINE)
    cleaned_res = re.sub(r'\s*```$', '', cleaned_res, flags=re.MULTILINE)
    
    try:
        data = json.loads(cleaned_res)
        return data
    except json.JSONDecodeError as e:
        print(f"Error when parsing JSON: {e}")

def read_code():
    dir = "../defect"
    codes = {}
    for filename in sorted(os.listdir(dir)):
        if filename.endswith('.ets'):
            with open(os.path.join(dir, filename), 'r', encoding='utf-8') as file:
                code = file.read()
                codes[filename.split('.')[0]] = code
    return codes

In [22]:
code = """
/*
 * Copyright (C) 2022 Huawei Device Co., Ltd.
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 *     http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */
import { Main } from '../Testbed/Framework/Main'
import * as box2d from '@ohos/box2d'
import { Settings } from '../Testbed/Framework/Test'
import { g_testEntries } from '../Testbed/Tests/TestEntries'

class ArrayItem {
  value: string = '';
}

@Entry
@Component
struct Index {
  private settings: RenderingContextSettings = new RenderingContextSettings(true)
  private context: CanvasRenderingContext2D = new CanvasRenderingContext2D(this.settings)
  private commonFontSize = 14;
  private fontColor = Color.Grey;
  private zindexRevers: boolean = false;
  private selectArr: Array<ArrayItem> = [];
  private selectIndex = 0;
  @State title: string = "Box2D Testbed version " + box2d.b2_version + "<br>(branch: " + box2d.b2_branch + " commit: " + box2d.b2_commit + ")";
  @State controlZindex: number = 99;
  @State canvasZindex: number = 1;
  private time = 60
  private app: Main = new Main(this.time, this.context)
  private m_settings = this.app ? this.app.m_settings : (new Settings())
  @State fps: string = this.app ? this.app.m_fps.toFixed(1).toString() : "FPS"
  @State index: number = this.app ? this.app.m_test_index : this.selectIndex
  private isLoad: boolean = false
  @State visibile: Visibility = Visibility.Visible;

  aboutToAppear() {
    for (let i: number = 0; i < g_testEntries.length; ++i) {
      let o: ArrayItem = new ArrayItem();
      o.value = g_testEntries[i].name;
      this.selectArr[i] = o;
    }

    const loop = () => {
      let tt = new Date().getTime()
      this.app.SimulationLoop(tt)
      setTimeout(loop, this.time)
    }
    setTimeout(loop, this.time)
  }

  build() {
    Stack() {
      Scroll() {
        Flex() {
          Column() {
            Text(this.title)
              .fontSize(16)
              .fontWeight(FontWeight.Bold)
              .fontColor(Color.Grey)

            Text(this.fps)
              .fontSize(16)
              .fontWeight(FontWeight.Bold)
              .fontColor(Color.Grey)

            Text("Tests")
              .fontSize(16)
              .fontWeight(FontWeight.Bold)
              .fontColor(Color.Grey)

            Select(this.selectArr)
              .space(8)
              .selected(this.index)
              .value(this.selectArr[this.index].value)
              .font({ size: 20, weight: 200, family: 'serif', style: FontStyle.Normal })
              .selectedOptionFont({ size: 30, weight: 300, family: 'serif', style: FontStyle.Normal })
              .optionFont({ size: 20, weight: 200, family: 'serif', style: FontStyle.Normal })
              .onSelect((index: number) => {
                this.index = index;
                if (this.app) {
                  this.app.m_test_index = index;
                  this.app.LoadTest();
                }
              })

            this.connect_number_input("Vel Iters", this.m_settings.velocityIterations, (value: number): void => {
              this.m_settings.velocityIterations = value;
            }, 1, 20, 1);
            this.connect_number_input("Pos Iters", this.m_settings.positionIterations, (value: number): void => {
              this.m_settings.positionIterations = value;
            }, 1, 20, 1);
            // #if B2_ENABLE_PARTICLE
            this.connect_number_input("Pcl Iters", this.m_settings.particleIterations, (value: number): void => {
              this.m_settings.particleIterations = value;
            }, 1, 100, 1);
            // #endif
            this.connect_number_input("Hertz", this.m_settings.hz, (value: number): void => {
              this.m_settings.hz = value;
            }, 10, 120, 1);


            this.connect_checkbox_input("Sleep", this.m_settings.enableSleep, (value: boolean): void => {
              this.m_settings.enableSleep = value;
            });
            this.connect_checkbox_input("Warm Starting", this.m_settings.enableWarmStarting, (value: boolean): void => {
              this.m_settings.enableWarmStarting = value;
            });
            this.connect_checkbox_input("Time of Impact", this.m_settings.enableContinuous, (value: boolean): void => {
              this.m_settings.enableContinuous = value;
            });
            this.connect_checkbox_input("Sub-Stepping", this.m_settings.enableSubStepping, (value: boolean): void => {
              this.m_settings.enableSubStepping = value;
            });
            // #if B2_ENABLE_PARTICLE
            this.connect_checkbox_input("Strict Particle/Body Contacts", this.m_settings.strictContacts, (value: boolean): void => {
              this.m_settings.strictContacts = value;
            });
            //    // #endif

            Text("Draw")
              .fontSize(16)
              .fontWeight(FontWeight.Bold)
              .fontColor(Color.Grey)

            this.connect_checkbox_input("Shapes", this.m_settings.drawShapes, (value: boolean): void => {
              this.m_settings.drawShapes = value;
            });
            // #if B2_ENABLE_PARTICLE
            this.connect_checkbox_input("Particles", this.m_settings.drawParticles, (value: boolean): void => {
              this.m_settings.drawParticles = value;
            });
            // #endif
            this.connect_checkbox_input("Joints", this.m_settings.drawJoints, (value: boolean): void => {
              this.m_settings.drawJoints = value;
            });
            this.connect_checkbox_input("AABBs", this.m_settings.drawAABBs, (value: boolean): void => {
              this.m_settings.drawAABBs = value;
            });
            this.connect_checkbox_input("Contact Points", this.m_settings.drawContactPoints, (value: boolean): void => {
              this.m_settings.drawContactPoints = value;
            });
            this.connect_checkbox_input("Contact Normals", this.m_settings.drawContactNormals, (value: boolean): void => {
              this.m_settings.drawContactNormals = value;
            });
            this.connect_checkbox_input("Contact Impulses", this.m_settings.drawContactImpulse, (value: boolean): void => {
              this.m_settings.drawContactImpulse = value;
            });
            this.connect_checkbox_input("Friction Impulses", this.m_settings.drawFrictionImpulse, (value: boolean): void => {
              this.m_settings.drawFrictionImpulse = value;
            });
            this.connect_checkbox_input("Center of Masses", this.m_settings.drawCOMs, (value: boolean): void => {
              this.m_settings.drawCOMs = value;
            });
            this.connect_checkbox_input("Statistics", this.m_settings.drawStats, (value: boolean): void => {
              this.m_settings.drawStats = value;
            });
            this.connect_checkbox_input("Profile", this.m_settings.drawProfile, (value: boolean): void => {
              this.m_settings.drawProfile = value;
            });


            this.connect_button_input("Pause (P)", (e: ClickEvent
            ): void => {
              this.app.Pause();
            });
            this.connect_button_input("Single Step (O)", (e: ClickEvent
            ): void => {
              this.app.SingleStep();
            });
            this.connect_button_input("Restart (R)", (e: ClickEvent
            ): void => {
              this.app.LoadTest();
            });
            this.connect_button_input("Demo", (e: ClickEvent
            ): void => {
              this.app.ToggleDemo();
            });
          }
          .width('100%')
        }

      }.zIndex(this.controlZindex)
      .visibility(this.visibile)

      Flex() {
        Canvas(this.context)
          .width('100%')
          .height('100%')
          .backgroundColor('#ffff00')
          .onReady(() => {
            this.isLoad = true
          })
      }
      .width('100%')
      .height('100%')
      .position({ x: 0, y: 0 })
      .zIndex(this.canvasZindex)
      .onClick((event: ClickEvent) => {

      })
      .onHover((isHover: boolean) => {

      })
      .onMouse((e: MouseEvent) => {
        switch (e.action) {
          case MouseAction.Press:
            this.app.HandleMouseDown(e);
            break;
          case MouseAction.Release:
            this.app.HandleMouseUp(e);
            break;
          case MouseAction.Move:
            this.app.HandleMouseMove(e);
            break;
          case MouseAction.Hover:
            this.app.HandleMouseWheel(e)
            break;
        }
      })
      .onTouch((e: TouchEvent) => {
        switch (e.type) {
          case TouchType.Down:
            this.app.HandleTouchStart(e);
            break;
          case TouchType.Up:
            this.app.HandleTouchEnd(e);
            break;
          case TouchType.Move:
            this.app.HandleTouchMove(e);
            break;
          case TouchType.Cancel:
            break;
        }
      })

      Flex() {
        Button('图层切换')
          .fontColor(Color.White)
          .margin({ left: '10', top: '10' })
          .padding({ left: 20, right: 20, bottom: 10, top: 10 })
          .fontSize(this.commonFontSize)
          .onClick((event: ClickEvent) => {
            this.zindexRevers = !this.zindexRevers;
            if (!this.zindexRevers) {
              this.controlZindex = 99;
              this.canvasZindex = 1;
              this.visibile = Visibility.Visible;
            } else {
              this.controlZindex = 1;
              this.canvasZindex = 99;
              this.visibile = Visibility.None;
            }

          });
      }.width("30%")
      .position({ x: -10, y: 70 })
      .zIndex(999)

    }.onKeyEvent((e: KeyEvent) => {
      switch (e.type) {
        case KeyType.Down:
          this.app.HandleKeyDown(e);
          break;
        case KeyType.Up:
          this.app.HandleKeyUp(e);
          break;
      }
    })

  }

  @Builder
  connect_number_input(label: string, init: number, update: (value: number) => void, min: number, max: number, step: number) {
    Flex() {
      Text(label)
        .textAlign(TextAlign.Center)
        .fontColor(this.fontColor)
        .margin({ left: '20' })
        .fontSize(this.commonFontSize);

      TextInput({
        text: init + ''
      })
        .width('30%')
        .margin({ left: '20' })
        .textAlign(TextAlign.Start)
        .fontColor(this.fontColor)
        .fontSize(this.commonFontSize)
        .onChange((value: string) => {
          update(Number.parseInt(value, 10));
        });
    }
  }

  @Builder
  connect_checkbox_input(label: string, init: boolean, update: (value: boolean) => void) {
    Flex() {
      Checkbox({ name: 'checkbox1', group: 'checkboxGroup' })
        .select(init)
        .selectedColor(0xed6f21)
        .onChange((value: boolean) => {
          update(value);
        })
      Text(label)
        .textAlign(TextAlign.Center)
        .fontColor(this.fontColor)
        .margin({ left: '10' })
        .fontSize(this.commonFontSize);
    }
  }

  @Builder
  connect_button_input(label: string, callback: (e: ClickEvent
  ) => void) {
    Flex() {
      Button(label)
        .fontColor(Color.White)
        .margin({ left: '10', top: '10' })
        .padding({ left: 20, right: 20, bottom: 10, top: 10 })
        .fontSize(this.commonFontSize)
        .onClick((event: ClickEvent) => {
          callback(event);
        });
    }
  }
}
"""


In [33]:
res = generate("Following is my arkts code which you should check whether there are defects: \n" + code, system_prompt=system_prompt)
vul_type_data = handle_vul_type_res(res) 
print(json.dumps(vul_type_data, indent=4))

[
    {
        "rule": "@performance/hp-arkui-use-reusable-component",
        "description": "\u5efa\u8bae\u590d\u6742\u7ec4\u4ef6\u7684\u5b9a\u4e49\uff0c\u5c3d\u91cf\u4f7f\u7528\u7ec4\u4ef6\u590d\u7528\u3002 \u6ed1\u52a8\u4e22\u5e27\u573a\u666f\u4e0b\uff0c\u5efa\u8bae\u4f18\u5148\u4fee\u6539\u3002",
        "line": 79,
        "defect_snippet": "Select(this.selectArr) \n  .space(8) \n  .selected(this.index) \n  .value(this.selectArr[this.index].value) \n  .font({ size: 20, weight: 200, family: 'serif', style: FontStyle.Normal }) \n  .selectedOptionFont({ size: 30, weight: 300, family: 'serif', style: FontStyle.Normal }) \n  .optionFont({ size: 20, weight: 200, family: 'serif', style: FontStyle.Normal }) \n  .onSelect((index: number) => { \n    this.index = index; \n    if (this.app) { \n      this.app.m_test_index = index; \n      this.app.LoadTest(); \n    } \n  })"
    },
    {
        "rule": "@performance/hp-arkui-no-state-var-access-in-loop",
        "description": "\u907f\u514

### 只识别一个缺陷

In [9]:
sys_prompt = """
    I will give you an arkts file, please determine whether there's defect in the code. If there exists defect, determine which type of defect exists in the code, and give the result in one of the following rules. If there're no defect, choose 'no defect':
    ['@performance/sparse-array-check', '@performance/hp-arkui-use-row-column-to-replace-flex', '@performance/hp-arkui-no-state-var-access-in-loop', '@performance/hp-arkui-use-reusable-component', '@performance/hp-arkui-avoid-empty-callback', '@performance/hp-arkui-no-stringify-in-lazyforeach-key-generator', '@performance/hp-arkui-load-on-demand', '@performance/hp-arkui-use-id-in-get-resource-sync-api', '@performance/hp-arkui-replace-nested-reusable-component-by-builder', '@performance/hp-arkui-use-onAnimationStart-for-swiper-preload', '@performance/hp-arkui-suggest-use-effectkit-blur', '@performance/hp-arkui-image-async-load', '@performance/number-init-check', '@performance/lottie-animation-destroy-check', '@performance/hp-arkui-no-func-as-arg-for-reusable-component', '@performance/multiple-associations-state-var-check', '@performance/hp-arkui-use-transition-to-replace-animateto', '@performance/typed-array-check', '@performance/hp-arkui-reduce-pan-gesture-distance', '@performance/foreach-args-check', '@performance/timezone-interface-check', '@performance/hp-arkui-use-attributeUpdater-control-refresh-scope', '@performance/hp-arkts-no-use-any-export-current', '@performance/hp-arkui-use-local-var-to-replace-state-var', '@performance/hp-arkui-use-word-break-to-replace-zero-width-space', '@performance/hp-arkui-suggest-cache-avplayer', '@performance/hp-arkui-remove-redundant-state-var', '@performance/constant-property-referencing-check-in-loops', '@performance/hp-arkui-combine-same-arg-animateto', '@performance/hp-performance-no-dynamic-cls-func', '@performance/hp-arkui-use-taskpool-for-web-request', '@performance/hp-arkui-use-scale-to-replace-attr-animateto', '@performance/no-high-loaded-frame-rate-range', '@performance/hp-arkui-suggest-reuseid-for-if-else-reusable-component', '@performance/hp-arkui-use-grid-layout-options', '@performance/hp-arkts-no-use-any-export-other', '@performance/waterflow-data-preload-check', '@performance/hp-arkui-remove-container-without-property', '@performance/hp-arkui-set-cache-count-for-lazyforeach-grid', '@performance/hp-arkui-suggest-use-get-anonymousid-async', '@performance/hp-arkui-remove-redundant-nest-container', '@performance/hp-arkui-remove-unchanged-state-var', '@performance/hp-performance-no-closures', '@performance/high-frequency-log-check', '@performance/hp-arkui-use-object-link-to-replace-prop', '@performance/hp-arkui-avoid-update-auto-state-var-in-aboutToReuse']

    Please output the result in json format, without giving any additional information. If the defect snippet contains a newline character, convert it to \\n. If there are multiple defects, choose the most important one to output. For example:

{
    "rule": "@performance/constant-property-referencing-check-in-loops",
    "description": "If you should frequently access a constant in a loop, and the property referencing the constant will not change in the loop, it is recommended to extract it outside the loop to reduce the number of property accesses",
    "line": [11, 12],
    "defect snippet": "for (let i = 0; i < arr.length; i++) {\\n    console.log(arr[1]);\\n}",
}

    Importantly, please try to recognize the above rules for why the code disobey the defect and what is the rule's meaning from the description. 
    When reading the code, please pay attention to the code which may disobey the following defect, and try to find the defect in the code.
"""

In [13]:
res = generate("Following is my arkts code which you should check whether there are defects: \n" + code, system_prompt=rules_prompt + sys_prompt)
vul_type_data = handle_vul_type_res(res) 
print(json.dumps(vul_type_data, indent=4))


{
    "rule": "@performance/hp-arkui-avoid-update-auto-state-var-in-aboutToReuse",
    "description": "Avoid updating auto-updated state variables in aboutToReuse.",
    "line": [
        132,
        134
    ],
    "defect snippet": "this.index = index;\nthis.app.m_test_index = index;"
}


In [10]:
# from pinecone import Pinecone
# import json
# import torch
# import uuid

# # 定义生成嵌入向量的函数
# def get_embedding(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
#     outputs = model(**inputs)
#     # 使用最后一个隐藏层的平均池化作为句子嵌入
#     embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
#     return embeddings

# def search_similar_defect(code):
#     query_vector = get_embedding(code)
#     # 初始化 Pinecone
#     pc = Pinecone(api_key="40075f49-8396-4571-924a-4b6d342cc81d")
#     index_name = "arkts-defects"
#     # 连接到索引
#     index = pc.Index(index_name)
#     # 检索最相似的向量
#     results = index.query(namespace="arkts", vector=query_vector.tolist(), top_k=10, include_metadata=True)
#     return results.matches

In [12]:
codes = read_code()

for filename, code in codes.items():
    print("-"*50)
    print(filename)
    res = generate("Following is my arkts code which you should check whether there are defects: \n" + code, model='gpt-4o-mini', system_prompt=rules_prompt + sys_prompt)
    vul_type_data = handle_vul_type_res(res) 
    print(vul_type_data)
    break

    ## double check the result whether the result is correct
    # res = generate([
    #     { "role": "user", "content": "Following is my arkts code which you should check whether there's vulnerability: \n" + code},
    #     { "role": "assistant", "content": res},
    #     { "role": "user", "content": "Please double check whether the result(the code has the defect which is recognized by the rule) is correct. Please output True or False in key 'result' in json format. If correct, please output the result in the same format as the example above. If not, please output the correct result in the same format as the example above."}
    # ], model='gpt-4o-mini', system_prompt=rules_prompt + sys_prompt)
    # print(res)


--------------------------------------------------
constant-property-referencing-check-in-loops
[{'rule': '@performance/constant-property-referencing-check-in-loops', 'description': 'If you should frequently access a constant in a loop, and the property referencing the constant will not change in the loop, it is recommended to extract it outside the loop to reduce the number of property accesses', 'line': [5, 7], 'defect snippet': 'total += (Settings.config[index] * i);'}]


In [ ]:
# codes = read_code()

# for filename, code in codes.items():
#     conversation = [
#         {"role": "user", "content": rules_prompt},
#         {"role": "assistant", "content": ruls_exp},
#         {"role": "user", "content": sys_prompt + "Following is my arkts code which you should check whether there's vulnerability: \n" + code},
#     ]

#     res = generate(conversation, base_url='https://api.deepseek.com', model='deepseek-coder')
#     print("-"*50 + '\n' + filename + ':\n' + res + '\n' + "-")
#     vul_type_data = handle_vul_type_res(res) 

In [ ]:
# files = ['hp-arkui-use-attributeUpdater-control-refresh-scope', 'hp-arkui-set-cache-count-for-lazyforeach-grid', 
#          'hp-performance-no-closures', 'hp-arkui-remove-redundant-nest-container',
#          'hp-arkts-no-use-any-export-current', 'hp-arkui-remove-unchanged-state-var', 'hp-arkui-remove-unchanged-state-var-2', 'hp-arkui-remove-redundant-state-var' ]

# for filename in files:
#     code = codes[filename]
#     conversation = [
#         {"role": "user", "content": rules_prompt},
#         {"role": "assistant", "content": ruls_exp},
#         {"role": "user", "content": sys_prompt + attention + "Following is my arkts code which you should check whether there's vulnerability: \n" + code},
#     ]

#     res = generate(conversation)
#     print("-"*50 + '\n' + filename + ':\n' + res + '\n' + "-"*50)
#     vul_type_data = handle_vul_type_res(res) 

## Repair